<a href="https://colab.research.google.com/github/skaumbdoallsaws-coder/AI-Drawing-Inspector/blob/main/ai_inspector_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI Engineering Drawing Inspector v2.0

**Single-File QC Pipeline**

Outputs:
1. `ResolvedPartIdentity.json`
2. `DrawingEvidence.json`
3. `DiffResult.json`
4. `QCReport.md`

In [ ]:
# Cell 1: Install Dependencies
!pip install -q pymupdf opencv-python-headless jsonschema pillow pytesseract
!pip install -q accelerate qwen-vl-utils bitsandbytes
!pip install -q git+https://github.com/huggingface/transformers
!pip install -q json-repair
!apt-get install -y poppler-utils tesseract-ocr > /dev/null 2>&1
print("Dependencies installed!")

In [ ]:
# Cell 2: Imports and Configuration
import os, json, re, gc
import torch
import fitz
import numpy as np
from PIL import Image
from pathlib import Path
from dataclasses import dataclass, field, asdict
from typing import List, Dict, Any, Optional, Tuple
from datetime import datetime

# Configuration
DRAWING_PDF_PATH = ""
SOLIDWORKS_JSON_DIR = "sw_json_library"
OUTPUT_DIR = "qc_output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")

In [ ]:
# Cell 3: BOM-Robust JSON Loader
def load_json_robust(filepath) -> Tuple[Optional[Dict], Optional[str]]:
    """Load JSON with BOM handling. Tries: utf-8-sig, utf-8, latin-1"""
    filepath = Path(filepath)
    for enc in ['utf-8-sig', 'utf-8', 'latin-1']:
        try:
            with open(filepath, 'r', encoding=enc) as f:
                return json.load(f), None
        except UnicodeDecodeError:
            continue
        except json.JSONDecodeError as e:
            if 'BOM' in str(e) and enc == 'utf-8':
                continue
            return None, f"JSON error: {str(e)[:50]}"
        except Exception as e:
            return None, f"Error: {str(e)[:50]}"
    return None, "Failed all encodings"

print("load_json_robust defined")

In [ ]:
# Cell 4: PDF Rendering
@dataclass
class PageArtifact:
    pageIndex0: int
    page: int
    image: Image.Image
    width: int
    height: int
    dpi: int
    direct_text: Optional[str] = None

def render_pdf(pdf_path: str, dpi: int = 300) -> List[PageArtifact]:
    """Render first page of PDF to image."""
    artifacts = []
    doc = fitz.open(pdf_path)
    page = doc.load_page(0)
    zoom = dpi / 72.0
    pix = page.get_pixmap(matrix=fitz.Matrix(zoom, zoom), alpha=False)
    img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
    direct_text = page.get_text("text")

    artifacts.append(PageArtifact(
        pageIndex0=0, page=1, image=img,
        width=pix.width, height=pix.height, dpi=dpi,
        direct_text=direct_text if len(direct_text.strip()) > 10 else None
    ))
    doc.close()
    print(f"Rendered: {pix.width}x{pix.height}px")
    return artifacts

print("render_pdf defined")

In [ ]:
# Cell 5: SolidWorks JSON Library
@dataclass
class SwPartEntry:
    json_path: str
    part_number: str
    filename_stem: str = ""
    data: Dict[str, Any] = field(default_factory=dict)

class SwJsonLibrary:
    def __init__(self):
        self.by_part_number: Dict[str, SwPartEntry] = {}
        self.by_filename: Dict[str, SwPartEntry] = {}
        self.all_entries: List[SwPartEntry] = []

    def _normalize(self, s: str) -> str:
        return re.sub(r'[-\s_]', '', str(s or '')).lower()

    def load_from_directory(self, directory: str):
        json_files = list(Path(directory).glob("**/*.json"))
        print(f"Found {len(json_files)} JSON files")

        for jp in json_files:
            data, err = load_json_robust(jp)
            if data is None:
                continue
            pn = data.get('identity', {}).get('partNumber', '')
            entry = SwPartEntry(str(jp), pn, jp.stem, data)
            self.all_entries.append(entry)
            if pn:
                self.by_part_number[pn] = entry
                self.by_part_number[self._normalize(pn)] = entry
            self.by_filename[jp.stem] = entry
            self.by_filename[self._normalize(jp.stem)] = entry
        print(f"Loaded {len(self.all_entries)} files")

    def lookup(self, candidate: str) -> Optional[SwPartEntry]:
        if not candidate:
            return None
        norm = self._normalize(candidate)
        return self.by_part_number.get(candidate) or self.by_part_number.get(norm) or \
               self.by_filename.get(candidate) or self.by_filename.get(norm)

sw_library = SwJsonLibrary()
print("SwJsonLibrary defined")

In [ ]:
# Cell 6: Part Identity Resolution (Robust Matching)

@dataclass
class ResolvedPartIdentity:
    partNumber: str
    confidence: float
    source: str
    swJsonPath: Optional[str] = None
    candidates_tried: List[str] = field(default_factory=list)

def clean_filename(filename: str) -> str:
    """Remove known suffixes like Paint, REV, etc."""
    cleaned = re.sub(r'[\s_]*(Paint|PAINT)$', '', filename, flags=re.IGNORECASE)
    return cleaned.strip()

def extract_pn_candidates(filename: str) -> List[str]:
    """
    Extract potential part number candidates from filename.
    Handles: 1013572_01, 101357201-03, 314884W_0, 046-935-REV-A
    Returns list of candidates (most specific to least).
    """
    name_no_ext = os.path.splitext(filename)[0]
    # Remove duplicate markers like (1), (2)
    name_no_ext = re.sub(r'\s*\(\d+\)$', '', name_no_ext)
    cleaned = clean_filename(name_no_ext)
    parts = re.split(r'[\s_]+', cleaned)

    if not parts:
        return []

    base = parts[0]
    candidates = []

    # 1. Base as-is
    candidates.append(base)

    # 2. Without hyphens
    base_no_hyphen = base.replace('-', '')
    if base_no_hyphen != base:
        candidates.append(base_no_hyphen)

    # 3. Remove letter suffixes (046-935A -> 046-935)
    if base and base[-1].isalpha() and len(base) > 1:
        candidates.append(base[:-1])
        candidates.append(base[:-1].replace('-', ''))

    # 4. Handle revision pattern (046-935-01 -> 046-935)
    rev_match = re.match(r'^(.+)-(\d{1,2})$', base)
    if rev_match:
        main_part = rev_match.group(1)
        candidates.append(main_part)
        candidates.append(main_part.replace('-', ''))

    # 5. Handle REV suffix (046-935-REV-A -> 046-935)
    rev_alpha = re.match(r'^(.+?)[-_]?REV[-_]?[A-Z0-9]*$', base, re.IGNORECASE)
    if rev_alpha:
        candidates.append(rev_alpha.group(1))
        candidates.append(rev_alpha.group(1).replace('-', ''))

    # 6. Peeling - progressively remove trailing digits
    temp = base_no_hyphen
    while len(temp) > 5:
        temp = temp[:-1]
        candidates.append(temp)

    # Remove duplicates, preserve order
    seen = set()
    unique = []
    for c in candidates:
        if c and c not in seen:
            seen.add(c)
            unique.append(c)

    return unique

def resolve_part_identity(pdf_path: str, artifacts: List[PageArtifact], sw_lib: SwJsonLibrary) -> ResolvedPartIdentity:
    """Resolve part identity using robust filename matching."""
    filename = os.path.basename(pdf_path)
    candidates = extract_pn_candidates(filename)

    # Try each candidate against SW library
    for candidate in candidates:
        entry = sw_lib.lookup(candidate)
        if entry:
            return ResolvedPartIdentity(
                partNumber=entry.part_number or candidate,
                confidence=1.0,
                source="filename+sw",
                swJsonPath=entry.json_path,
                candidates_tried=candidates
            )

    # Try PDF embedded text
    for art in artifacts:
        if art.direct_text:
            text_candidates = extract_pn_candidates(art.direct_text[:200])
            for candidate in text_candidates[:5]:
                entry = sw_lib.lookup(candidate)
                if entry:
                    return ResolvedPartIdentity(
                        partNumber=entry.part_number or candidate,
                        confidence=0.8,
                        source="pdf_text+sw",
                        swJsonPath=entry.json_path,
                        candidates_tried=candidates + text_candidates[:5]
                    )

    # Fallback - use first candidate or filename stem
    fallback_pn = candidates[0] if candidates else Path(pdf_path).stem
    return ResolvedPartIdentity(
        partNumber=fallback_pn,
        confidence=0.3,
        source="fallback",
        swJsonPath=None,
        candidates_tried=candidates
    )

print("resolve_part_identity defined (robust matching)")

In [ ]:
# Cell 7: Load SolidWorks Library (Upload ZIP)
from google.colab import files
import zipfile

if not os.path.exists(SOLIDWORKS_JSON_DIR) or not list(Path(SOLIDWORKS_JSON_DIR).glob("*.json")):
    print("Upload your sw_json_library.zip file:")
    uploaded = files.upload()

    for filename in uploaded:
        if filename.endswith('.zip'):
            print(f"Extracting {filename}...")
            with zipfile.ZipFile(filename, 'r') as z:
                z.extractall(SOLIDWORKS_JSON_DIR)
            print(f"Extracted to {SOLIDWORKS_JSON_DIR}")
            break

sw_library.load_from_directory(SOLIDWORKS_JSON_DIR)
print(f"Library ready: {len(sw_library.all_entries)} parts indexed")

In [ ]:
# Cell 8: Upload and Render PDF Drawing
from google.colab import files
from IPython.display import display

print("Upload your PDF drawing:")
uploaded = files.upload()

for filename in uploaded:
    if filename.lower().endswith('.pdf'):
        DRAWING_PDF_PATH = filename
        break

print(f"Processing: {DRAWING_PDF_PATH}")
artifacts = render_pdf(DRAWING_PDF_PATH)

# Display the rendered image
if artifacts:
    display(artifacts[0].image.resize((800, int(800 * artifacts[0].height / artifacts[0].width))))

In [ ]:
# Cell 9: Resolve Part Identity
part_identity = resolve_part_identity(DRAWING_PDF_PATH, artifacts, sw_library)

print("="*50)
print("RESOLVED PART IDENTITY")
print("="*50)
print(f"Part Number:  {part_identity.partNumber}")
print(f"Confidence:   {part_identity.confidence}")
print(f"Source:       {part_identity.source}")
print(f"SW JSON:      {part_identity.swJsonPath or 'Not found'}")
print(f"Candidates:   {part_identity.candidates_tried[:5]}")

# Save to output
identity_out = os.path.join(OUTPUT_DIR, "ResolvedPartIdentity.json")
with open(identity_out, 'w') as f:
    json.dump(asdict(part_identity), f, indent=2)
print(f"\nSaved: {identity_out}")

In [23]:
# Cell 10: Load LightOnOCR-2 and Run OCR
from transformers import LightOnOcrForConditionalGeneration, LightOnOcrProcessor
from google.colab import userdata

# Clear GPU memory
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()

# Get HF token
try:
    hf_token = userdata.get('HF_TOKEN')
except:
    hf_token = None

print("Loading LightOnOCR-2-1B...")
ocr_device = "cuda" if torch.cuda.is_available() else "cpu"
ocr_dtype = torch.bfloat16 if ocr_device == "cuda" else torch.float32

ocr_processor = LightOnOcrProcessor.from_pretrained(
    "lightonai/LightOnOCR-2-1B",
    token=hf_token
)

ocr_model = LightOnOcrForConditionalGeneration.from_pretrained(
    "lightonai/LightOnOCR-2-1B",
    torch_dtype=ocr_dtype,
    token=hf_token
).to(ocr_device)

print(f"LightOnOCR-2 loaded: {ocr_model.get_memory_footprint() / 1e9:.2f} GB")

def run_lighton_ocr(image: Image.Image) -> List[str]:
    """Run LightOnOCR-2 on image, return list of text lines."""
    global ocr_model, ocr_processor, ocr_device, ocr_dtype

    img = image.convert("RGB")
    conversation = [{"role": "user", "content": [{"type": "image", "image": img}]}]

    inputs = ocr_processor.apply_chat_template(
        conversation, add_generation_prompt=True, tokenize=True,
        return_dict=True, return_tensors="pt"
    )
    inputs = {k: v.to(device=ocr_device, dtype=ocr_dtype) if v.is_floating_point() else v.to(ocr_device) for k, v in inputs.items()}

    with torch.no_grad():
        output_ids = ocr_model.generate(**inputs, max_new_tokens=2048)

    generated_ids = output_ids[0, inputs["input_ids"].shape[1]:]
    output_text = ocr_processor.decode(generated_ids, skip_special_tokens=True)

    return [line.strip() for line in output_text.split("\n") if line.strip()]

# Run OCR on the drawing
print("Running OCR on drawing...")
ocr_lines = run_lighton_ocr(artifacts[0].image)
print(f"OCR extracted {len(ocr_lines)} lines")
print("\nFirst 10 lines:")
for line in ocr_lines[:10]:
    print(f"  {line}")

Loading LightOnOCR-2-1B...


You are using a model of type mistral3 to instantiate a model of type lighton_ocr. This is not supported for all configurations of models and can yield errors.


Loading weights:   0%|          | 0/532 [00:00<?, ?it/s]

LightOnOCR-2 loaded: 2.02 GB
Running OCR on drawing...
OCR extracted 64 lines

First 10 lines:
  M6X1.0–6H THRU (a)
  4.00±.03
  1.00±.03
  (d)
  .50±.03
  .25±.03
  .25±.03
  .75±.03
  3.50
  2X φ.281 THRU (d)


In [24]:
# Cell 10b: Qwen2.5-VL Drawing Understanding (with JSON repair)
from transformers import AutoModelForVision2Seq, AutoProcessor
from qwen_vl_utils import process_vision_info
from json_repair import repair_json

# Clear some GPU memory before loading Qwen
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()

print("Loading Qwen2.5-VL-7B for drawing understanding...")
qwen_model_id = "Qwen/Qwen2.5-VL-7B-Instruct"

qwen_processor = AutoProcessor.from_pretrained(qwen_model_id, trust_remote_code=True)
qwen_model = AutoModelForVision2Seq.from_pretrained(
    qwen_model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True
)
print(f"Qwen2.5-VL loaded: {qwen_model.get_memory_footprint() / 1e9:.2f} GB")

def analyze_drawing_with_qwen(image: Image.Image) -> Dict[str, Any]:
    """Use Qwen2.5-VL to understand the engineering drawing visually."""

    prompt = """Analyze this engineering drawing and identify all features. Return a JSON object with:

{
  "partDescription": "brief description of the part",
  "views": ["list of views shown: TOP, FRONT, SIDE, ISOMETRIC, SECTION, DETAIL"],
  "features": [
    {
      "type": "TappedHole|ThroughHole|BlindHole|Counterbore|Countersink|Slot|Fillet|Chamfer|Thread",
      "description": "brief description",
      "callout": "the dimension/callout text if visible",
      "quantity": 1,
      "location": "where on the part"
    }
  ],
  "material": "material if shown in title block",
  "titleBlockInfo": {
    "partNumber": "if visible",
    "revision": "if visible",
    "scale": "if visible"
  },
  "notes": ["any general notes visible on drawing"]
}

Be thorough - identify ALL holes, threads, chamfers, fillets, and other machined features you can see.
Only return valid JSON, no other text."""

    messages = [
        {
            "role": "user",
            "content": [
                {"type": "image", "image": image},
                {"type": "text", "text": prompt}
            ]
        }
    ]

    text = qwen_processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    image_inputs, video_inputs = process_vision_info(messages)

    inputs = qwen_processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt"
    ).to(qwen_model.device)

    with torch.no_grad():
        output_ids = qwen_model.generate(**inputs, max_new_tokens=4096, temperature=0.1)

    generated_ids = output_ids[0, inputs.input_ids.shape[1]:]
    response = qwen_processor.decode(generated_ids, skip_special_tokens=True)

    # Parse JSON from response with repair
    try:
        # Try to extract JSON block from response
        json_match = re.search(r'```json\s*([\s\S]*?)\s*```', response)
        if json_match:
            json_str = json_match.group(1)
        else:
            json_match = re.search(r'\{[\s\S]*\}', response)
            json_str = json_match.group() if json_match else response

        # Try direct parse first
        try:
            return json.loads(json_str)
        except json.JSONDecodeError:
            # Use json-repair to fix malformed JSON
            print("  Attempting JSON repair...")
            repaired = repair_json(json_str)
            return json.loads(repaired)

    except Exception as e:
        return {"raw_response": response[:1000], "parse_error": str(e)}

# Analyze the drawing
print("Analyzing drawing with Qwen2.5-VL...")
qwen_understanding = analyze_drawing_with_qwen(artifacts[0].image)

print("="*50)
print("QWEN2.5-VL DRAWING UNDERSTANDING")
print("="*50)

if "parse_error" not in qwen_understanding:
    print(f"Part: {qwen_understanding.get('partDescription', 'N/A')}")
    print(f"Views: {qwen_understanding.get('views', [])}")
    print(f"Material: {qwen_understanding.get('material', 'N/A')}")
    print(f"\nFeatures identified: {len(qwen_understanding.get('features', []))}")
    for f in qwen_understanding.get('features', [])[:10]:
        print(f"  - {f.get('type')}: {f.get('callout', f.get('description', ''))}")
    if qwen_understanding.get('notes'):
        print(f"\nNotes: {qwen_understanding.get('notes', [])[:3]}")
else:
    print(f"Parse error: {qwen_understanding.get('parse_error')}")
    print(f"Raw response:\n{qwen_understanding.get('raw_response', '')[:500]}")

# Save understanding
understanding_out = os.path.join(OUTPUT_DIR, "QwenUnderstanding.json")
with open(understanding_out, 'w') as f:
    json.dump(qwen_understanding, f, indent=2)
print(f"\nSaved: {understanding_out}")

ImportError: cannot import name 'AutoModelForVision2Seq' from 'transformers' (/usr/local/lib/python3.12/dist-packages/transformers/__init__.py)

In [25]:
# Cell 11: Merge OCR + Qwen Understanding into Enriched Evidence

INCH_TO_MM = 25.4

# Regex patterns for extracting callouts from OCR
PATTERNS = {
    'metric_thread': r'M(\d+(?:\.\d+)?)\s*[xX]\s*(\d+(?:\.\d+)?)',
    'imperial_thread': r'(\d+/\d+)\s*-\s*(\d+)',
    'thru_hole': r'[oOØ∅φ]?\s*(\.?\d+\.?\d*|\d+)\s*(?:mm|MM|")?\s*THRU',
    'blind_hole': r'[oOØ∅φ]?\s*(\.?\d+\.?\d*|\d+)\s*[xX]\s*(\d+\.?\d*)\s*(?:DEEP|DP)',
    'fillet': r'\bR(\d+\.?\d*)\b',
    'chamfer': r'(\d+\.?\d*)\s*[xX]\s*45\s*[°]?',
}

def is_likely_imperial(value: float, raw_text: str) -> bool:
    if '"' in raw_text or 'IN' in raw_text.upper():
        return True
    if value < 1.0 and 'mm' not in raw_text.lower():
        return True
    imperial_sizes = [0.125, 0.1875, 0.25, 0.281, 0.3125, 0.375, 0.4375, 0.5, 0.625, 0.75, 0.875]
    for imp in imperial_sizes:
        if abs(value - imp) < 0.01:
            return True
    return False

def convert_to_mm(value: float, raw_text: str) -> float:
    if is_likely_imperial(value, raw_text):
        return round(value * INCH_TO_MM, 3)
    return value

def parse_ocr_callouts(ocr_lines: List[str]) -> List[Dict]:
    """Extract callouts from OCR text."""
    callouts = []
    raw_text = "\n".join(ocr_lines)

    for match in re.finditer(PATTERNS['metric_thread'], raw_text, re.IGNORECASE):
        callouts.append({
            'calloutType': 'TappedHole',
            'thread': {'standard': 'Metric', 'nominalDiameterMm': float(match.group(1)), 'pitch': float(match.group(2))},
            'raw': match.group(0), 'source': 'ocr'
        })

    for match in re.finditer(PATTERNS['thru_hole'], raw_text, re.IGNORECASE):
        raw = match.group(0)
        val = float(match.group(1))
        callouts.append({
            'calloutType': 'Hole', 'diameterMm': convert_to_mm(val, raw), 'diameterRaw': val,
            'isImperial': is_likely_imperial(val, raw), 'isThrough': True, 'raw': raw, 'source': 'ocr'
        })

    for match in re.finditer(PATTERNS['fillet'], raw_text, re.IGNORECASE):
        raw = match.group(0)
        val = float(match.group(1))
        callouts.append({'calloutType': 'Fillet', 'radiusMm': convert_to_mm(val, raw), 'raw': raw, 'source': 'ocr'})

    for match in re.finditer(PATTERNS['chamfer'], raw_text, re.IGNORECASE):
        raw = match.group(0)
        val = float(match.group(1))
        callouts.append({'calloutType': 'Chamfer', 'distance1Mm': convert_to_mm(val, raw), 'angleDegrees': 45, 'raw': raw, 'source': 'ocr'})

    return callouts

def parse_qwen_features(qwen_data: Dict) -> List[Dict]:
    """Convert Qwen features to callout format."""
    callouts = []
    if "parse_error" in qwen_data:
        return callouts

    type_map = {
        'TappedHole': 'TappedHole', 'ThroughHole': 'Hole', 'BlindHole': 'Hole',
        'Counterbore': 'Hole', 'Countersink': 'Hole', 'Fillet': 'Fillet',
        'Chamfer': 'Chamfer', 'Thread': 'TappedHole', 'Slot': 'Slot'
    }

    for feat in qwen_data.get('features', []):
        ftype = feat.get('type', '')
        callout_type = type_map.get(ftype, ftype)
        callout = feat.get('callout', '')
        qty = feat.get('quantity', 1)

        entry = {
            'calloutType': callout_type,
            'description': feat.get('description', ''),
            'location': feat.get('location', ''),
            'quantity': qty,
            'raw': callout,
            'source': 'qwen'
        }

        # Try to parse dimensions from Qwen's callout text
        if callout:
            thread_match = re.search(r'M(\d+(?:\.\d+)?)[xX](\d+(?:\.\d+)?)', callout)
            if thread_match:
                entry['thread'] = {
                    'standard': 'Metric',
                    'nominalDiameterMm': float(thread_match.group(1)),
                    'pitch': float(thread_match.group(2))
                }

            hole_match = re.search(r'[oOØ∅φ]?\s*(\.?\d+\.?\d*)', callout)
            if hole_match and callout_type == 'Hole':
                val = float(hole_match.group(1))
                entry['diameterMm'] = convert_to_mm(val, callout)
                entry['diameterRaw'] = val
                entry['isThrough'] = 'THRU' in callout.upper()

        callouts.append(entry)

    return callouts

def merge_evidence(ocr_callouts: List[Dict], qwen_callouts: List[Dict]) -> List[Dict]:
    """Merge OCR and Qwen callouts, preferring OCR for dimensions but using Qwen for context."""
    merged = []
    used_qwen = set()

    for ocr in ocr_callouts:
        merged_entry = ocr.copy()
        merged_entry['sources'] = ['ocr']

        # Try to find matching Qwen feature for additional context
        for qi, qwen in enumerate(qwen_callouts):
            if qi in used_qwen:
                continue
            if ocr.get('calloutType') == qwen.get('calloutType'):
                # Check if dimensions roughly match
                if ocr.get('thread') and qwen.get('thread'):
                    if ocr['thread'].get('nominalDiameterMm') == qwen['thread'].get('nominalDiameterMm'):
                        merged_entry['location'] = qwen.get('location', '')
                        merged_entry['description'] = qwen.get('description', '')
                        merged_entry['sources'].append('qwen')
                        used_qwen.add(qi)
                        break
                elif ocr.get('diameterMm') and qwen.get('diameterMm'):
                    if abs(ocr['diameterMm'] - qwen['diameterMm']) < 1.0:
                        merged_entry['location'] = qwen.get('location', '')
                        merged_entry['description'] = qwen.get('description', '')
                        merged_entry['sources'].append('qwen')
                        used_qwen.add(qi)
                        break

        merged.append(merged_entry)

    # Add Qwen features not matched to OCR (may be features OCR missed)
    for qi, qwen in enumerate(qwen_callouts):
        if qi not in used_qwen:
            qwen['sources'] = ['qwen_only']
            merged.append(qwen)

    return merged

# Parse both sources
ocr_callouts = parse_ocr_callouts(ocr_lines)
qwen_callouts = parse_qwen_features(qwen_understanding)

# Merge
merged_callouts = merge_evidence(ocr_callouts, qwen_callouts)

# Build enriched evidence
evidence = {
    'schemaVersion': '1.2.0',
    'partNumber': part_identity.partNumber,
    'extractedAt': datetime.now().isoformat() + 'Z',
    'sources': {
        'ocr': {'model': 'LightOnOCR-2-1B', 'lineCount': len(ocr_lines)},
        'vision': {'model': 'Qwen2-VL-7B', 'featureCount': len(qwen_callouts)}
    },
    'drawingInfo': {
        'views': qwen_understanding.get('views', []),
        'partDescription': qwen_understanding.get('partDescription', ''),
        'material': qwen_understanding.get('material', ''),
        'titleBlock': qwen_understanding.get('titleBlockInfo', {}),
        'notes': qwen_understanding.get('notes', [])
    },
    'foundCallouts': merged_callouts,
    'rawOcrSample': ocr_lines[:15]
}

print("="*50)
print("MERGED DRAWING EVIDENCE")
print("="*50)
print(f"OCR callouts:   {len(ocr_callouts)}")
print(f"Qwen features:  {len(qwen_callouts)}")
print(f"Merged total:   {len(merged_callouts)}")
print(f"\nDrawing info:")
print(f"  Views: {evidence['drawingInfo']['views']}")
print(f"  Material: {evidence['drawingInfo']['material']}")
print(f"\nMerged callouts:")
for c in merged_callouts[:10]:
    sources = '+'.join(c.get('sources', []))
    extra = f" [{sources}]"
    if c.get('location'):
        extra += f" @ {c['location']}"
    print(f"  {c['calloutType']}: {c.get('raw', c.get('description', ''))}{extra}")

# Save
evidence_out = os.path.join(OUTPUT_DIR, "DrawingEvidence.json")
with open(evidence_out, 'w') as f:
    json.dump(evidence, f, indent=2)
print(f"\nSaved: {evidence_out}")

NameError: name 'qwen_understanding' is not defined

In [22]:
# Cell 12: Generate DiffResult (Fixed: uses comparison.holeGroups)

def extract_sw_requirements(sw_data: Dict) -> List[Dict]:
    """Extract requirements from SolidWorks JSON using comparison.holeGroups."""
    requirements = []

    # Primary source: comparison.holeGroups (reconciled/canonical data)
    comparison = sw_data.get('comparison', {})
    hole_groups = comparison.get('holeGroups', [])

    for hg in hole_groups:
        hole_type = hg.get('holeType', '')
        canonical = hg.get('canonical', '')
        count = hg.get('count', 1)
        diameters = hg.get('diameters', {})
        thread = hg.get('thread', {})

        if hole_type == 'Tapped':
            # Tapped hole - extract thread info
            requirements.append({
                'type': 'TappedHole',
                'thread': {
                    'standard': thread.get('standard', 'Metric'),
                    'nominalDiameterMm': thread.get('majorDiameterMm') or diameters.get('threadNominalDiameterMm'),
                    'pitch': thread.get('pitch'),
                    'callout': thread.get('callout', canonical)
                },
                'count': count,
                'canonical': canonical,
                'source': 'sw_comparison.holeGroups'
            })
        elif hole_type == 'Through':
            # Plain through hole
            diameter_mm = diameters.get('pilotOrTapDrillDiameterMm')
            requirements.append({
                'type': 'Hole',
                'diameterMm': diameter_mm,
                'diameterInches': diameters.get('pilotOrTapDrillDiameterInches'),
                'isThrough': True,
                'count': count,
                'canonical': canonical,
                'source': 'sw_comparison.holeGroups'
            })
        elif hole_type == 'Blind':
            # Blind hole
            diameter_mm = diameters.get('pilotOrTapDrillDiameterMm')
            requirements.append({
                'type': 'Hole',
                'diameterMm': diameter_mm,
                'isThrough': False,
                'count': count,
                'canonical': canonical,
                'source': 'sw_comparison.holeGroups'
            })

    # Fallback: features.holeWizardHoles if no comparison data
    if not requirements:
        features = sw_data.get('features', {})
        for hole in features.get('holeWizardHoles', []):
            if hole.get('isTapped'):
                thread_size = hole.get('threadSize', '')
                # Parse M6x1.0 format
                m = re.match(r'M(\d+(?:\.\d+)?)[xX](\d+(?:\.\d+)?)', thread_size)
                if m:
                    requirements.append({
                        'type': 'TappedHole',
                        'thread': {
                            'standard': 'Metric',
                            'nominalDiameterMm': float(m.group(1)),
                            'pitch': float(m.group(2)),
                            'callout': thread_size
                        },
                        'count': hole.get('instanceCount', 1),
                        'source': 'sw_features.holeWizardHoles'
                    })
            else:
                requirements.append({
                    'type': 'Hole',
                    'diameterMm': hole.get('diameter', 0) * 1000,  # meters to mm
                    'isThrough': hole.get('isThrough', False),
                    'count': hole.get('instanceCount', 1),
                    'source': 'sw_features.holeWizardHoles'
                })

        # Fillets
        for fillet in features.get('fillets', []):
            requirements.append({
                'type': 'Fillet',
                'radiusMm': fillet.get('radius'),
                'source': 'sw_features'
            })

        # Chamfers
        for chamfer in features.get('chamfers', []):
            requirements.append({
                'type': 'Chamfer',
                'distance1Mm': chamfer.get('distance1'),
                'angleDegrees': chamfer.get('angle', 45),
                'source': 'sw_features'
            })

    return requirements

def compare_callout_to_requirement(callout: Dict, req: Dict, tolerance: float = 0.5) -> bool:
    """Check if a drawing callout matches a SW requirement."""
    ctype = callout.get('calloutType')
    rtype = req.get('type')

    if ctype != rtype:
        return False

    if ctype == 'Hole':
        d1 = callout.get('diameterMm', 0)
        d2 = req.get('diameterMm', 0)
        if d1 and d2 and abs(d1 - d2) <= tolerance:
            return True

    elif ctype == 'TappedHole':
        t1 = callout.get('thread', {})
        t2 = req.get('thread', {})
        # Match by nominal diameter (within 0.1mm tolerance)
        nom1 = t1.get('nominalDiameterMm', 0)
        nom2 = t2.get('nominalDiameterMm', 0)
        if nom1 and nom2 and abs(nom1 - nom2) < 0.1:
            # Also check pitch if available
            p1 = t1.get('pitch')
            p2 = t2.get('pitch')
            if p1 and p2:
                return abs(p1 - p2) < 0.01
            return True

    elif ctype == 'Fillet':
        r1 = callout.get('radiusMm', 0)
        r2 = req.get('radiusMm', 0)
        if r1 and r2 and abs(r1 - r2) <= tolerance:
            return True

    elif ctype == 'Chamfer':
        d1 = callout.get('distance1Mm', 0)
        d2 = req.get('distance1Mm', 0)
        if d1 and d2 and abs(d1 - d2) <= tolerance:
            return True

    return False

def generate_diff_result(evidence: Dict, sw_data: Dict) -> Dict:
    """Compare drawing evidence against SolidWorks requirements."""
    callouts = evidence.get('foundCallouts', [])
    requirements = extract_sw_requirements(sw_data)

    found = []
    missing = []
    matched_callouts = set()
    matched_requirements = set()

    # Check each requirement against callouts
    for ri, req in enumerate(requirements):
        match_found = False
        for ci, callout in enumerate(callouts):
            if ci not in matched_callouts and compare_callout_to_requirement(callout, req):
                found.append({
                    'status': 'FOUND',
                    'requirement': req,
                    'evidence': callout,
                    'note': f"Matched: {req.get('canonical', req.get('type'))}"
                })
                matched_callouts.add(ci)
                matched_requirements.add(ri)
                match_found = True
                break

        if not match_found:
            missing.append({
                'status': 'MISSING',
                'requirement': req,
                'evidence': None,
                'note': f"Not found in drawing: {req.get('canonical', req.get('type'))}"
            })

    # Extra callouts not matched to any requirement
    extra = []
    for ci, callout in enumerate(callouts):
        if ci not in matched_callouts:
            extra.append({
                'status': 'EXTRA',
                'requirement': None,
                'evidence': callout,
                'note': f"In drawing but not in SW: {callout.get('raw', callout.get('calloutType'))}"
            })

    diff_result = {
        'partNumber': evidence.get('partNumber'),
        'generatedAt': datetime.now().isoformat() + 'Z',
        'summary': {
            'totalRequirements': len(requirements),
            'found': len(found),
            'missing': len(missing),
            'extra': len(extra),
            'matchRate': f"{len(found)/len(requirements)*100:.1f}%" if requirements else "N/A"
        },
        'details': {
            'found': found,
            'missing': missing,
            'extra': extra
        }
    }

    return diff_result

# Load SW data and generate diff
if part_identity.swJsonPath:
    sw_data, err = load_json_robust(part_identity.swJsonPath)
    if sw_data:
        # Show what we're extracting
        requirements = extract_sw_requirements(sw_data)
        print("="*50)
        print("SW REQUIREMENTS EXTRACTED")
        print("="*50)
        for req in requirements:
            print(f"  {req['type']}: {req.get('canonical', req.get('thread', {}).get('callout', ''))}")

        diff_result = generate_diff_result(evidence, sw_data)

        print("\n" + "="*50)
        print("DIFF RESULT")
        print("="*50)
        print(f"Part: {diff_result['partNumber']}")
        print(f"  Total Requirements: {diff_result['summary']['totalRequirements']}")
        print(f"  FOUND:   {diff_result['summary']['found']}")
        print(f"  MISSING: {diff_result['summary']['missing']}")
        print(f"  EXTRA:   {diff_result['summary']['extra']}")
        print(f"  Match Rate: {diff_result['summary']['matchRate']}")

        if diff_result['details']['found']:
            print("\nMatched:")
            for item in diff_result['details']['found']:
                print(f"  ✓ {item['note']}")

        if diff_result['details']['missing']:
            print("\nMissing from drawing:")
            for item in diff_result['details']['missing']:
                print(f"  ✗ {item['note']}")

        if diff_result['details']['extra']:
            print("\nExtra in drawing:")
            for item in diff_result['details']['extra']:
                print(f"  ? {item['note']}")

        # Save
        diff_out = os.path.join(OUTPUT_DIR, "DiffResult.json")
        with open(diff_out, 'w') as f:
            json.dump(diff_result, f, indent=2)
        print(f"\nSaved: {diff_out}")
    else:
        print(f"Error loading SW JSON: {err}")
else:
    print("No SW JSON path - cannot generate diff")

SW REQUIREMENTS EXTRACTED
  TappedHole: M6x1.0
  TappedHole: M5x0.8 (7X)
  Hole: ø7.10mm THRU (2X)


NameError: name 'evidence' is not defined